# Algorithmic Trading System - Example Usage

This notebook demonstrates how to use the algorithmic trading system for strategy development, backtesting, and analysis.

In [ ]:
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Import our modules
from src.data.data_fetcher import DataFetcher
from src.data.preprocessor import DataPreprocessor
from src.strategies.moving_average_strategy import MovingAverageStrategy
from src.strategies.rsi_strategy import RSIStrategy
from src.strategies.macd_strategy import MACDStrategy
from src.backtesting.backtest_engine import BacktestEngine
from src.risk_management.risk_manager import RiskManager
from src.visualization.visualizer import TradingVisualizer

## 1. Data Fetching and Preprocessing

In [ ]:
# Configuration
SYMBOL = 'AAPL'
START_DATE = '2022-01-01'
END_DATE = '2023-12-31'
INITIAL_CAPITAL = 100000

# Fetch data
data_fetcher = DataFetcher()
market_data = data_fetcher.fetch_market_data([SYMBOL], START_DATE, END_DATE)

# Display basic info
if SYMBOL in market_data:
    data = market_data[SYMBOL]
    print(f"Data shape: {data.shape}")
    print(f"Date range: {data.index[0]} to {data.index[-1]}")
    print(f"\nFirst few rows:")
    display(data.head())
else:
    print("No data retrieved")

In [ ]:
# Preprocess data
preprocessor = DataPreprocessor()
processed_data = preprocessor.process_multiple_symbols(
    market_data,
    add_technical_indicators=True,
    add_features=True
)

# Display processed data info
processed = processed_data[SYMBOL]
print(f"Processed data shape: {processed.shape}")
print(f"\nTechnical indicators added:")
technical_cols = [col for col in processed.columns if col not in ['Open', 'High', 'Low', 'Close', 'Volume']]
print(technical_cols[:10])  # Show first 10

## 2. Strategy Development and Signal Generation

In [ ]:
# Initialize strategies
ma_strategy = MovingAverageStrategy(short_window=20, long_window=50)
rsi_strategy = RSIStrategy(rsi_period=14, oversold_threshold=30, overbought_threshold=70)
macd_strategy = MACDStrategy()

# Set symbol attribute for signal generation
symbol_data = processed_data[SYMBOL].copy()
symbol_data.attrs['symbol'] = SYMBOL

# Generate signals
ma_signals = ma_strategy.generate_signals(symbol_data)
rsi_signals = rsi_strategy.generate_signals(symbol_data)
macd_signals = macd_strategy.generate_signals(symbol_data)

print(f"Moving Average signals: {len(ma_signals)}")
print(f"RSI signals: {len(rsi_signals)}")
print(f"MACD signals: {len(macd_signals)}")

# Display sample signals
if ma_signals:
    print(f"\nSample MA signal:")
    sample_signal = ma_signals[0]
    print(f"Date: {sample_signal.timestamp}")
    print(f"Signal: {sample_signal.signal.name}")
    print(f"Price: ${sample_signal.price:.2f}")
    print(f"Confidence: {sample_signal.confidence:.2f}")

## 3. Backtesting

In [ ]:
# Initialize backtest engine
backtest_engine = BacktestEngine(initial_capital=INITIAL_CAPITAL)

# Run backtests for each strategy
strategies = {
    'Moving Average': {SYMBOL: ma_signals},
    'RSI': {SYMBOL: rsi_signals},
    'MACD': {SYMBOL: macd_signals}
}

results = {}

for strategy_name, signals in strategies.items():
    print(f"\nBacktesting {strategy_name}...")
    
    result = backtest_engine.run_backtest(
        market_data=processed_data,
        signals=signals,
        start_date=START_DATE,
        end_date=END_DATE
    )
    
    results[strategy_name] = result
    
    # Print metrics
    metrics = result.metrics
    print(f"Total Return: {metrics.get('Total_Return_Pct', 0):.2f}%")
    print(f"Sharpe Ratio: {metrics.get('Sharpe_Ratio', 0):.2f}")
    print(f"Max Drawdown: {metrics.get('Max_Drawdown_Pct', 0):.2f}%")
    print(f"Win Rate: {metrics.get('Win_Rate_Pct', 0):.1f}%")
    print(f"Total Trades: {metrics.get('Total_Trades', 0)}")

## 4. Performance Analysis

In [ ]:
# Compare strategy performance
performance_df = pd.DataFrame({
    strategy_name: {
        'Total Return (%)': result.metrics.get('Total_Return_Pct', 0),
        'Sharpe Ratio': result.metrics.get('Sharpe_Ratio', 0),
        'Max Drawdown (%)': result.metrics.get('Max_Drawdown_Pct', 0),
        'Win Rate (%)': result.metrics.get('Win_Rate_Pct', 0),
        'Total Trades': result.metrics.get('Total_Trades', 0),
        'Profit Factor': result.metrics.get('Profit_Factor', 0)
    }
    for strategy_name, result in results.items()
}).T

print("Strategy Performance Comparison:")
display(performance_df.round(2))

In [ ]:
# Plot portfolio values over time
plt.figure(figsize=(12, 6))

for strategy_name, result in results.items():
    if not result.portfolio_values.empty:
        portfolio_returns = (result.portfolio_values / INITIAL_CAPITAL - 1) * 100
        plt.plot(portfolio_returns.index, portfolio_returns.values, 
                label=strategy_name, linewidth=2)

# Add buy and hold benchmark
benchmark_returns = (processed_data[SYMBOL]['Close'] / processed_data[SYMBOL]['Close'].iloc[0] - 1) * 100
plt.plot(benchmark_returns.index, benchmark_returns.values, 
         label='Buy & Hold', linestyle='--', alpha=0.7)

plt.title(f'Strategy Performance Comparison - {SYMBOL}')
plt.xlabel('Date')
plt.ylabel('Return (%)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Risk Analysis

In [ ]:
# Risk analysis
risk_manager = RiskManager()

# Select best performing strategy for detailed risk analysis
best_strategy = max(results.items(), 
                   key=lambda x: x[1].metrics.get('Total_Return_Pct', -999))

strategy_name, result = best_strategy
print(f"Risk Analysis for Best Strategy: {strategy_name}")
print("=" * 50)

if not result.portfolio_values.empty:
    # Calculate risk metrics
    portfolio_returns = result.portfolio_values.pct_change().dropna()
    
    # Create position data for risk analysis
    positions = {'Portfolio': result.portfolio_values.iloc[-1] - INITIAL_CAPITAL}
    returns_data = {'Portfolio': portfolio_returns}
    
    risk_metrics = risk_manager.calculate_portfolio_risk(positions, returns_data)
    risk_level = risk_manager.assess_risk_level(risk_metrics)
    
    print(f"Risk Level: {risk_level.name}")
    print(f"Value at Risk (95%): {risk_metrics.var_95:.4f}")
    print(f"Conditional VaR (95%): {risk_metrics.cvar_95:.4f}")
    print(f"Volatility: {risk_metrics.volatility:.4f}")
    print(f"Beta: {risk_metrics.beta:.4f}")
    print(f"Max Drawdown: {risk_metrics.max_drawdown:.4f}")
    
    # Risk recommendations
    risk_checks = risk_manager.check_risk_limits(positions, result.portfolio_values.iloc[-1], 0)
    recommendations = risk_manager.generate_risk_recommendations(risk_metrics, risk_level, risk_checks)
    
    if recommendations:
        print("\nRisk Recommendations:")
        for rec in recommendations:
            print(f"• {rec}")
    else:
        print("\nNo specific risk recommendations.")

## 6. Trade Analysis

In [ ]:
# Analyze individual trades
trades_df = result.to_dataframe()

if not trades_df.empty:
    print(f"Trade Analysis for {strategy_name}:")
    print("=" * 40)
    
    # Basic trade statistics
    winning_trades = trades_df[trades_df['PnL'] > 0]
    losing_trades = trades_df[trades_df['PnL'] < 0]
    
    print(f"Total Trades: {len(trades_df)}")
    print(f"Winning Trades: {len(winning_trades)} ({len(winning_trades)/len(trades_df)*100:.1f}%)")
    print(f"Losing Trades: {len(losing_trades)} ({len(losing_trades)/len(trades_df)*100:.1f}%)")
    print(f"Average Win: ${winning_trades['PnL'].mean():.2f}" if len(winning_trades) > 0 else "Average Win: $0.00")
    print(f"Average Loss: ${losing_trades['PnL'].mean():.2f}" if len(losing_trades) > 0 else "Average Loss: $0.00")
    print(f"Largest Win: ${trades_df['PnL'].max():.2f}")
    print(f"Largest Loss: ${trades_df['PnL'].min():.2f}")
    print(f"Average Trade Duration: {trades_df['Duration_Days'].mean():.1f} days")
    
    # Plot trade PnL distribution
    plt.figure(figsize=(10, 4))
    
    plt.subplot(1, 2, 1)
    plt.hist(trades_df['PnL'], bins=20, alpha=0.7, edgecolor='black')
    plt.axvline(x=0, color='red', linestyle='--', alpha=0.7)
    plt.title('Trade PnL Distribution')
    plt.xlabel('PnL ($)')
    plt.ylabel('Frequency')
    
    plt.subplot(1, 2, 2)
    cumulative_pnl = trades_df['PnL'].cumsum()
    plt.plot(range(len(cumulative_pnl)), cumulative_pnl, linewidth=2)
    plt.title('Cumulative PnL')
    plt.xlabel('Trade Number')
    plt.ylabel('Cumulative PnL ($)')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Show recent trades
    print("\nRecent Trades:")
    display(trades_df.tail(10))
else:
    print("No trades to analyze.")

## 7. Strategy Optimization

In [ ]:
# Parameter optimization example
print("Optimizing Moving Average Strategy Parameters...")

# Define parameter ranges for optimization
parameter_ranges = {
    'short_window': [10, 15, 20, 25],
    'long_window': [40, 50, 60, 70]
}

# Optimize parameters
ma_strategy_opt = MovingAverageStrategy()
best_params, best_score = ma_strategy_opt.optimize_parameters(
    symbol_data,
    parameter_ranges,
    optimization_metric='sharpe_ratio'
)

print(f"\nBest Parameters: {best_params}")
print(f"Best Sharpe Ratio: {best_score:.3f}")

# Compare with default parameters
default_signals = MovingAverageStrategy().generate_signals(symbol_data)
optimized_signals = ma_strategy_opt.generate_signals(symbol_data)

print(f"\nSignal Comparison:")
print(f"Default Strategy Signals: {len(default_signals)}")
print(f"Optimized Strategy Signals: {len(optimized_signals)}")

## 8. Interactive Visualizations

In [ ]:
# Create interactive visualizations
visualizer = TradingVisualizer()

# Price and signals chart
if ma_signals:
    # Convert signals to DataFrame for visualization
    signal_df = pd.DataFrame([{
        'Date': signal.timestamp,
        'Signal': signal.signal.value,
        'Price': signal.price
    } for signal in ma_signals])
    signal_df.set_index('Date', inplace=True)
    
    # Create price and signals chart
    fig = visualizer.plot_price_and_signals(
        processed_data[SYMBOL],
        signal_df,
        SYMBOL,
        indicators=['SMA_20', 'SMA_50', 'RSI']
    )
    fig.show()

# Portfolio performance chart
if not result.portfolio_values.empty:
    portfolio_fig = visualizer.plot_portfolio_performance(
        result.portfolio_values,
        benchmark=processed_data[SYMBOL]['Close'],
        trades=trades_df
    )
    portfolio_fig.show()

## 9. Summary and Next Steps

In [ ]:
print("TRADING SYSTEM ANALYSIS SUMMARY")
print("=" * 50)

# Find best strategy
best_strategy_name = max(results.items(), 
                        key=lambda x: x[1].metrics.get('Total_Return_Pct', -999))[0]
best_metrics = results[best_strategy_name].metrics

print(f"Symbol Analyzed: {SYMBOL}")
print(f"Analysis Period: {START_DATE} to {END_DATE}")
print(f"Initial Capital: ${INITIAL_CAPITAL:,}")
print(f"\nBest Performing Strategy: {best_strategy_name}")
print(f"Total Return: {best_metrics.get('Total_Return_Pct', 0):.2f}%")
print(f"Sharpe Ratio: {best_metrics.get('Sharpe_Ratio', 0):.2f}")
print(f"Max Drawdown: {best_metrics.get('Max_Drawdown_Pct', 0):.2f}%")
print(f"Win Rate: {best_metrics.get('Win_Rate_Pct', 0):.1f}%")
print(f"Total Trades: {best_metrics.get('Total_Trades', 0)}")

print("\nNext Steps:")
print("• Experiment with different parameter combinations")
print("• Test on different time periods and market conditions")
print("• Implement ensemble methods combining multiple strategies")
print("• Add machine learning-based signal generation")
print("• Implement real-time trading capabilities")
print("• Add more sophisticated risk management rules")